In [1]:
import matplotlib.pylab as plt
import sys, os
import numpy as np
import pprint
import pandas as pd
from itertools import compress
%matplotlib inline

from tensorboard.backend.event_processing import event_accumulator as ea
from tensorboard.backend.event_processing import event_multiplexer as eam

from scipy.interpolate import spline
from scipy import ndimage
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [2]:

def highlight_max(data, color='yellow'):
    '''
    highlight the maximum in a Series or DataFrame
    '''
    attr = 'background-color: {}'.format(color)
    #remove % and cast to float
    data = data.replace('%','', regex=True).astype(float)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_max = data == data.max()
        return [attr if v else '' for v in is_max]
    else:  # from .apply(axis=None)
        is_max = data == data.max().max()
        return pd.DataFrame(np.where(is_max, attr, ''),
                            index=data.index, columns=data.columns)

def highlight_min(data, color='red'):
    '''
    highlight the minimum in a Series or DataFrame
    '''
    attr = 'background-color: {}'.format(color)
    #remove % and cast to float
    data = data.replace('%','', regex=True).astype(float)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_max = data == data.min()
        return [attr if v else '' for v in is_max]
    else:  # from .apply(axis=None)
        is_max = data == data.min().min()
        return pd.DataFrame(np.where(is_max, attr, ''),
                            index=data.index, columns=data.columns)
    
def highlight_min_max(data, colors=['red','blue']):
    '''
    highlight the extremes in a Series or DataFrame
    '''
    attrmin = 'color: {}'.format(colors[0])
    attrmax = 'color: {}'.format(colors[1])
    #remove % and cast to float
    data = data.replace('%','', regex=True).astype(float)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_max = data == data.max()
        is_min = data == data.min()
        attributes = np.where(is_max, attrmax, '')
        attributes = np.where(is_min, attrmin, attributes)
        return attributes
#         return [attrmax if ma elif mi '' for (ma,mi) in zip(is_max,is_min)]
    else:  # from .apply(axis=None)
        is_min = data == data.min().min()
        is_max = data == data.max().max()
        attributes = np.where(is_max, attrmax, '')
        attributes = np.where(is_min, attrmin, atributes)
        return pd.DataFrame(attributes,
                            index=data.index, columns=data.columns)
    
def get_summaries(x, run, tag):
    outcome = [(s.step, s.value) for s in x.Scalars(run,tag)]
    outcome = np.array(outcome)
    return outcome[:,0], outcome[:,1]

def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

def add_plot_sum(x, run, tag, sigma_smooth = 1, **kwargs):
    step, val = get_summaries(x, run,tag)
    plt.plot(step,val,  'r-', **kwargs)
    smooth_val = ndimage.gaussian_filter1d(val, sigma_smooth)
    #smooth_step = ndimage.gaussian_filter1d(step, sigma_smooth)
    if False: ## use tag as a label
        plt.plot(step,smooth_val,  'r-', label = tag, alpha = 1, color = kwargs['color'])
    else:
        plt.plot(step,smooth_val,  'r-', label = run, alpha = 1, color = kwargs['color'])

colors = ['blue', 'green', 'goldenrod', 'purple', 'royalblue','lime','cyan','magenta','red', ]


save_path='/home/andresro/iccv2019/tables'

lowermodelnames=['MODELcount_PATCH16_16_SCALE4_CH12_Lr0.5_Lsr1.0_Lw1.0_hrlab_sq2', #'MODELcount_PATCH16_16_SCALE4_CH12_Lr0.5_Lsr1.0_Lw1.0LOWER_hrlab_sq2',             
                 'MODELcount_PATCH16_16_SCALE2_CH12_Lr0.5_Lsr0.0_Lw1.0LOWER_sq2',
                 'MODELcount_PATCH16_16_SCALE2_CH12_Lr0.5_Lsr0.0_Lw1.0LOWER_hrlab_sq2', # 'MODELcount_PATCH16_16_SCALE2_CH12_Lr0.5_Lsr0.0_Lw1.0_hrlab_sq2after1'
                ]

save_tex = False


In [3]:
def create_ds(names_,sum_dirs_,lam_r, variables_= None):
    
    if lam_r == '1.0':
        if variables_ is None: variables_=['metrics/mae', 'metrics/mse']
        func_ = np.argmin
        colors = ['blue','red']
    elif lam_r == '0.0':
        if variables_ is None: variables_=['metrics/iou', 'metrics/acc', 'metrics/prec']
        func_ = np.argmax
        colors = ['red','blue']
    else:
        raise ValueError('lam_r={} not defined'.format(lam_r))
    
    df =pd.DataFrame({'Name':names_}).drop_duplicates(keep=False)
    new = df["Name"].str.split("_", n = 3, expand = True) 
    df.set_index('Name')
    df["Object"]= new[0] 
    df["Area"]= new[1].astype(np.float)
    df["Tag"]= new[2] 

    x1 = eam.EventMultiplexer({key: value for (key, value) in zip(names_,sum_dirs_) if key in d_(names_)})
    x1.Reload()

#     variables = ['metrics/iou'] #, 'metrics/mse', 'metrics/recall']

    for key in x1.Runs().keys():
        step = None
        for i, var_ in enumerate(variables_):
            x_ = x1.Scalars(key,var_)
            step = func_([i.value for i in x_]) if step is None else step
            x = x_[step]
            if not var_ in df.keys() or np.isnan(df[df.Name == key][var_]).any():
                # TODO add if key already exists
                df.loc[df.Name == key,var_] = x.value
        df.loc[df.Name == key,'step'] = x.step

    df['id1'] = df['Object']+'_'+df['Tag']
    d1 = df.sort_values(by=['Area']).pivot(index='id1', columns='Area', values=variables_)

    def process_index(k):
        return tuple(k.split("_"))
    d1.index = pd.MultiIndex.from_tuples([process_index(k) for k,v in d1.iterrows()])

    if save_tex:
        latex_table = d1.to_latex(float_format=lambda x: '' if np.isnan(x) else '%.3f' % x)
        with open(tablefile, "w") as text_file:
            text_file.write(latex_table)

    return d1.style.apply(lambda x:highlight_min_max(x,colors=colors))

## Baselines
Different models are trained for each of the tasks
1. SR0: super resolution architecture without SR task ($\lambda_{SR}=0$). High res labels
2. count: only LR input and labels
3. counth: downsampled HR to 10m and LR input, LR inputs

### Classification

In [12]:
path='/home/pf/pfstaff/projects/andresro/sparse/training_leon/snapshots'
names = ['100','10','2','1']
tablefile=save_path+"/results_baselines_c.tex"
names1 = []
sum_dirs = []
obj=['coco']

model= 'MODELcount_PATCH16_16_SCALE8_CH12_Lr0.0_Lsr0.0_Lw1.0_hrlab_sq2'
tag='count'
names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])

model= 'MODELcounth_PATCH16_16_SCALE8_CH12_Lr0.0_Lsr0.0_Lw1.0_hrlab_sq2'
tag='counth'
names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])

model= 'MODELcountSR_PATCH16_16_SCALE8_CH12_Lr0.0_Lsr0.0_Lw1.0_hrlab_sq2'
tag='SR0'
names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])


index = [os.path.isdir(dir_) for dir_ in sum_dirs]
print(np.sum(index), len(index))
d_ = lambda x: list(dict.fromkeys(x))

names1 = list(compress(names1, index))
sum_dirs=list(compress(sum_dirs,index))

data = create_ds(names1,sum_dirs,lam_r='0.0')
data


(12, 12)


In [13]:
path='/home/pf/pfstaff/projects/andresro/sparse/training_leon/snapshots'
names = ['100','10','2','1']
tablefile=save_path+"/results_baselines_c.tex"
names1 = []
sum_dirs = []
obj=['palm']

model= 'MODELcount_PATCH16_16_SCALE8_CH12_Lr0.0_Lsr0.0_Lw1.0_hrlab_sq2'
tag='count'
names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])

model= 'MODELcounth_PATCH16_16_SCALE8_CH12_Lr0.0_Lsr0.0_Lw1.0_hrlab_sq2'
tag='counth'
names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])

model= 'MODELcountSR_PATCH16_16_SCALE8_CH12_Lr0.0_Lsr0.0_Lw1.0_hrlab_sq2'
tag='SR0'
names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])


index = [os.path.isdir(dir_) for dir_ in sum_dirs]
print(np.sum(index), len(index))
d_ = lambda x: list(dict.fromkeys(x))

names1 = list(compress(names1, index))
sum_dirs=list(compress(sum_dirs,index))

data = create_ds(names1,sum_dirs,lam_r='0.0')
data


(6, 12)


### Regression

In [5]:
path='/home/pf/pfstaff/projects/andresro/sparse/training_leon/snapshots'
names = ['100','10','2','1']
tablefile=save_path+"/results_baselines_r.tex"
names1 = []
sum_dirs = []
obj=['coco']

model= 'MODELcount_PATCH16_16_SCALE8_CH12_Lr1.0_Lsr0.0_Lw1.0_hrlab_sq2'
tag='count'
names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])

model= 'MODELcounth_PATCH16_16_SCALE8_CH12_Lr1.0_Lsr0.0_Lw1.0_hrlab_sq2'
tag='counth'
names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])

model= 'MODELcountSR_PATCH16_16_SCALE8_CH12_Lr1.0_Lsr0.0_Lw1.0_hrlab_sq2'
tag='SR0'
names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])


index = [os.path.isdir(dir_) for dir_ in sum_dirs]
print(np.sum(index), len(index))
d_ = lambda x: list(dict.fromkeys(x))

names1 = list(compress(names1, index))
sum_dirs=list(compress(sum_dirs,index))

data = create_ds(names1,sum_dirs,lam_r='1.0')
data
# [(x, y) for x,y in zip(names1,sum_dirs) if x in d_(names1)]

(12, 12)


# Domain Adaption Models

### Simmilarity models
1. Rev: Reverse Gradient Layer with a discriminator between embeddings

1. L1: L1 loss on the embeddings

1. RevL: Both Rev and L1 loss on the embeddings

### Type of embeddings:
1. e: early layers

2. m: mid layers

3. l: before last layer

### Scales:
Scale of the HR data {8,16, 32}

## Rev
### Classification

In [6]:
path='/home/pf/pfstaff/projects/andresro/sparse/training_leon/snapshots'
names = ['100','10','2','1']
tablefile=save_path+"/results_domain.tex"
lam_r='0.0'
names1 = []
sum_dirs = []
obj=['coco']

for s in [8, 16, 32]:
    model= 'MODELcountDAearly_PATCH16_16_SCALE{}_CH12_Lr{}_Lsr0.0_Lw1.0_domainRev_sq2'.format(s,lam_r)
    tag='Rev{}e'.format(s)
    names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
    sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])

    model= 'MODELcountDAmid_PATCH16_16_SCALE{}_CH12_Lr{}_Lsr0.0_Lw1.0_domainRev_sq2'.format(s,lam_r)
    tag='Rev{}m'.format(s)
    names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
    sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])

    model= 'MODELcountDAlate_PATCH16_16_SCALE{}_CH12_Lr{}_Lsr0.0_Lw1.0_domainRev_sq2'.format(s,lam_r)
    tag='Rev{}l'.format(s)
    names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
    sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])

model= 'MODELcount_PATCH16_16_SCALE8_CH12_Lr{}_Lsr0.0_Lw1.0_hrlab_sq2'.format(lam_r)
tag='count'
names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])

index = [os.path.isdir(dir_) for dir_ in sum_dirs]
print(np.sum(index), len(index))
d_ = lambda x: list(dict.fromkeys(x))

names1 = list(compress(names1, index))
sum_dirs=list(compress(sum_dirs,index))

data = create_ds(names1,sum_dirs,lam_r)
data

(40, 40)


### Regression

In [7]:
path='/home/pf/pfstaff/projects/andresro/sparse/training_leon/snapshots'
names = ['100','10','2','1']
tablefile=save_path+"/results_domain_c.tex"
lam_r='1.0'
names1 = []
sum_dirs = []
obj=['coco']

for s in [8, 16, 32]:
    model= 'MODELcountDAearly_PATCH16_16_SCALE{}_CH12_Lr{}_Lsr0.0_Lw1.0_domainRev_sq2'.format(s,lam_r)
    tag='Rev{}e'.format(s)
    names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
    sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])

    model= 'MODELcountDAmid_PATCH16_16_SCALE{}_CH12_Lr{}_Lsr0.0_Lw1.0_domainRev_sq2'.format(s,lam_r)
    tag='Rev{}m'.format(s)
    names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
    sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])

    model= 'MODELcountDAlate_PATCH16_16_SCALE{}_CH12_Lr{}_Lsr0.0_Lw1.0_domainRev_sq2'.format(s,lam_r)
    tag='Rev{}l'.format(s)
    names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
    sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])

model= 'MODELcount_PATCH16_16_SCALE8_CH12_Lr{}_Lsr0.0_Lw1.0_hrlab_sq2'.format(lam_r)
tag='count'
names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])

index = [os.path.isdir(dir_) for dir_ in sum_dirs]
print(np.sum(index), len(index))
d_ = lambda x: list(dict.fromkeys(x))

names1 = list(compress(names1, index))
sum_dirs=list(compress(sum_dirs,index))

data = create_ds(names1,sum_dirs,lam_r)
data

(40, 40)


## Domain Adaption RevL

### Classification

In [8]:
path='/home/pf/pfstaff/projects/andresro/sparse/training_leon/snapshots'
names = ['100','10','2','1']
tablefile=save_path+"/results_domain.tex"
lam_r='0.0'
names1 = []
sum_dirs = []
obj=['coco']

for s in [8, 16, 32]:
    model= 'MODELcountDAearly_PATCH16_16_SCALE{}_CH12_Lr{}_Lsr0.0_Lw1.0_domainRevL_sq2'.format(s,lam_r)
    tag='RevL{}e'.format(s)
    names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
    sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])

    model= 'MODELcountDAmid_PATCH16_16_SCALE{}_CH12_Lr{}_Lsr0.0_Lw1.0_domainRevL_sq2'.format(s,lam_r)
    tag='RevL{}m'.format(s)
    names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
    sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])

    model= 'MODELcountDAlate_PATCH16_16_SCALE{}_CH12_Lr{}_Lsr0.0_Lw1.0_domainRevL_sq2'.format(s,lam_r)
    tag='RevL{}l'.format(s)
    names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
    sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])

model= 'MODELcount_PATCH16_16_SCALE8_CH12_Lr{}_Lsr0.0_Lw1.0_hrlab_sq2'.format(lam_r)
tag='count'
names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])

index = [os.path.isdir(dir_) for dir_ in sum_dirs]
print(np.sum(index), len(index))
d_ = lambda x: list(dict.fromkeys(x))

names1 = list(compress(names1, index))
sum_dirs=list(compress(sum_dirs,index))

data = create_ds(names1,sum_dirs,lam_r)
data

(40, 40)


### Regression

In [9]:
path='/home/pf/pfstaff/projects/andresro/sparse/training_leon/snapshots'
names = ['100','10','2','1']
tablefile=save_path+"/results_domain.tex"
lam_r='1.0'
names1 = []
sum_dirs = []
obj=['coco']

for s in [8, 16, 32]:
    model= 'MODELcountDAearly_PATCH16_16_SCALE{}_CH12_Lr{}_Lsr0.0_Lw1.0_domainRevL_sq2'.format(s,lam_r)
    tag='RevL{}e'.format(s)
    names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
    sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])

    model= 'MODELcountDAmid_PATCH16_16_SCALE{}_CH12_Lr{}_Lsr0.0_Lw1.0_domainRevL_sq2'.format(s,lam_r)
    tag='RevL{}m'.format(s)
    names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
    sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])

    model= 'MODELcountDAlate_PATCH16_16_SCALE{}_CH12_Lr{}_Lsr0.0_Lw1.0_domainRevL_sq2'.format(s,lam_r)
    tag='RevL{}l'.format(s)
    names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
    sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])

model= 'MODELcount_PATCH16_16_SCALE8_CH12_Lr{}_Lsr0.0_Lw1.0_hrlab_sq2'.format(lam_r)
tag='count'
names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])

index = [os.path.isdir(dir_) for dir_ in sum_dirs]
print(np.sum(index), len(index))
d_ = lambda x: list(dict.fromkeys(x))

names1 = list(compress(names1, index))
sum_dirs=list(compress(sum_dirs,index))

data = create_ds(names1,sum_dirs,lam_r)
data

(40, 40)


## Domain L1

### Classification

In [10]:
path='/home/pf/pfstaff/projects/andresro/sparse/training_leon/snapshots'
names = ['100','10','2','1']
tablefile=save_path+"/results_domain.tex"
lam_r='0.0'
names1 = []
sum_dirs = []
obj=['coco']

for s in [8, 16, 32]:
    model= 'MODELcountDAearly_PATCH16_16_SCALE{}_CH12_Lr{}_Lsr0.0_Lw1.0_domainL1_sq2'.format(s,lam_r)
    tag='L1{}e'.format(s)
    names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
    sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])

    model= 'MODELcountDAmid_PATCH16_16_SCALE{}_CH12_Lr{}_Lsr0.0_Lw1.0_domainL1_sq2'.format(s,lam_r)
    tag='L1{}m'.format(s)
    names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
    sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])

    model= 'MODELcountDAlate_PATCH16_16_SCALE{}_CH12_Lr{}_Lsr0.0_Lw1.0_domainL1_sq2'.format(s,lam_r)
    tag='L1{}l'.format(s)
    names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
    sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])

model= 'MODELcount_PATCH16_16_SCALE8_CH12_Lr{}_Lsr0.0_Lw1.0_hrlab_sq2'.format(lam_r)
tag='count'
names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])

index = [os.path.isdir(dir_) for dir_ in sum_dirs]
print(np.sum(index), len(index))
d_ = lambda x: list(dict.fromkeys(x))

names1 = list(compress(names1, index))
sum_dirs=list(compress(sum_dirs,index))

data = create_ds(names1,sum_dirs,lam_r)
data

(40, 40)


### Regression

In [11]:
path='/home/pf/pfstaff/projects/andresro/sparse/training_leon/snapshots'
names = ['100','10','2','1']
tablefile=save_path+"/results_domain.tex"
lam_r='1.0'
names1 = []
sum_dirs = []
obj=['coco']

for s in [8, 16, 32]:
    model= 'MODELcountDAearly_PATCH16_16_SCALE{}_CH12_Lr{}_Lsr0.0_Lw1.0_domainL1_sq2'.format(s,lam_r)
    tag='L1{}e'.format(s)
    names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
    sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])

    model= 'MODELcountDAmid_PATCH16_16_SCALE{}_CH12_Lr{}_Lsr0.0_Lw1.0_domainL1_sq2'.format(s,lam_r)
    tag='L1{}m'.format(s)
    names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
    sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])

    model= 'MODELcountDAlate_PATCH16_16_SCALE{}_CH12_Lr{}_Lsr0.0_Lw1.0_domainL1_sq2'.format(s,lam_r)
    tag='L1{}l'.format(s)
    names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
    sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])

model= 'MODELcount_PATCH16_16_SCALE8_CH12_Lr{}_Lsr0.0_Lw1.0_hrlab_sq2'.format(lam_r)
tag='count'
names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])

index = [os.path.isdir(dir_) for dir_ in sum_dirs]
print(np.sum(index), len(index))
d_ = lambda x: list(dict.fromkeys(x))

names1 = list(compress(names1, index))
sum_dirs=list(compress(sum_dirs,index))

data = create_ds(names1,sum_dirs,lam_r)
data

(40, 40)
